# Pandas: the Python structured data library

Pandas (allegedly) stands for **Pan**el **da**ta (**s**?) and lets you manipulate 'spreadsheet-like' data in Python easily

In [1]:
import pandas as pd

## Series: kind of like a `list` and `dict` put together

In [5]:
s = pd.Series([1,2,3])
s

0    1
1    2
2    3
dtype: int64

In [7]:
s = pd.Series([1,2,3], index='a b c'.split())
s

a    1
b    2
c    3
dtype: int64

In [8]:
s[0]

1

In [10]:
s['a']

1

## DataFrame -- the main data type



In [12]:
df = pd.DataFrame(
    [
        [1,2,3],
        [4,5,6],
        [7,8,9],
    ],
    columns='a b c'.split(),
    index='x y z'.split()
)
df

,a,b,c
x,1,2,3
y,4,5,6
z,7,8,9


In [13]:
df['a']

x    1
y    4
z    7
Name: a, dtype: int64

Multiple columns

In [20]:
df[['a', 'b', 'b']]

,a,b,b
x,1,2,2
y,4,5,5
z,7,8,8


# Indexing using .loc, .iloc

In [14]:
df.loc['x']

a    1
b    2
c    3
Name: x, dtype: int64

In [15]:
df.iloc[0]

a    1
b    2
c    3
Name: x, dtype: int64

In [16]:
df.loc['x', 'a']

1

In [18]:
df.loc['x', :]  # retrieve all columns

a    1
b    2
c    3
Name: x, dtype: int64

In [21]:
df.loc[:, 'a']  # retrieve all rows

x    1
y    4
z    7
Name: a, dtype: int64

# Reading CSV data

Most of the time, we *won't* be building `DataFrame`s out of the basic constructor, but rather using one of the readers built in to Pandas. One of these is `read_csv`:

In [25]:
df = pd.read_csv('./data/closing-prices.csv')
df.head() # Only show the first few rows

,Unnamed: 0,F,TSLA,GOOG,IBM,AAPL
0,2014-01-02,12.0890,150.10,NaN,157.6001,72.7741
1,2014-01-03,12.1438,149.56,NaN,158.5430,71.1756
2,2014-01-06,12.1986,147.00,NaN,157.9993,71.5637
3,2014-01-07,12.0420,149.36,NaN,161.1508,71.0516
4,2014-01-08,12.1673,151.28,NaN,159.6728,71.5019


The CSV reader is pretty good about inferring types, but not perfect. We can check lots of things about the structure of a `DataFrame` with the `.info()` method:

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 6 columns):
Unnamed: 0    1007 non-null object
F             1007 non-null float64
TSLA          1007 non-null float64
GOOG          949 non-null float64
IBM           1007 non-null float64
AAPL          1007 non-null float64
dtypes: float64(5), object(1)
memory usage: 47.3+ KB


The first column was read in as an `object` (meaning Pandas couldn't be more specific about its type, usually what happens with string data). Let's tell Pandas that column is a date:

In [27]:
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 6 columns):
Unnamed: 0    1007 non-null datetime64[ns]
F             1007 non-null float64
TSLA          1007 non-null float64
GOOG          949 non-null float64
IBM           1007 non-null float64
AAPL          1007 non-null float64
dtypes: datetime64[ns](1), float64(5)
memory usage: 47.3 KB


We can also parse datetimes during the import:

In [29]:
df = pd.read_csv('./data/closing-prices.csv', parse_dates=[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 6 columns):
Unnamed: 0    1007 non-null datetime64[ns]
F             1007 non-null float64
TSLA          1007 non-null float64
GOOG          949 non-null float64
IBM           1007 non-null float64
AAPL          1007 non-null float64
dtypes: datetime64[ns](1), float64(5)
memory usage: 47.3 KB


We can set the index of the dataframe as well:

In [30]:
df = df.set_index('Unnamed: 0')
df.head()

,F,TSLA,GOOG,IBM,AAPL
Unnamed: 0,,,,,
2014-01-02,12.0890,150.10,NaN,157.6001,72.7741
2014-01-03,12.1438,149.56,NaN,158.5430,71.1756
2014-01-06,12.1986,147.00,NaN,157.9993,71.5637
2014-01-07,12.0420,149.36,NaN,161.1508,71.0516
2014-01-08,12.1673,151.28,NaN,159.6728,71.5019


Its even better if we do it when we read in the frame:

In [31]:
df = pd.read_csv('./data/closing-prices.csv', index_col=0, parse_dates=[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1007 entries, 2014-01-02 to 2017-12-29
Data columns (total 5 columns):
F       1007 non-null float64
TSLA    1007 non-null float64
GOOG    949 non-null float64
IBM     1007 non-null float64
AAPL    1007 non-null float64
dtypes: float64(5)
memory usage: 47.2 KB


In [32]:
df.head()

,F,TSLA,GOOG,IBM,AAPL
2014-01-02,12.0890,150.10,NaN,157.6001,72.7741
2014-01-03,12.1438,149.56,NaN,158.5430,71.1756
2014-01-06,12.1986,147.00,NaN,157.9993,71.5637
2014-01-07,12.0420,149.36,NaN,161.1508,71.0516
2014-01-08,12.1673,151.28,NaN,159.6728,71.5019


## Reading from external APIs

There are some data sources for market data available in the pandas_datareader package:

In [37]:
from datetime import datetime

import pandas_datareader.data as web

start, end = datetime(2014, 1, 1), datetime(2018, 1, 1)
data = web.DataReader(
    ['F', 'TSLA', 'GOOG', 'IBM', 'AAPL', 'CRM'], 
    'yahoo', start, end,
)
data.head()

Attributes  Adj Close                                                 \
Symbols             F        TSLA        GOOG         IBM       AAPL   
Date                                                                   
2014-01-02  11.349146  150.100006  554.481689  144.451340  70.938835   
2014-01-03  11.400599  149.559998  550.436829  145.315567  69.380615   
2014-01-06  11.452051  147.000000  556.573853  144.817307  69.758965   
2014-01-07  11.305044  149.360001  567.303589  147.705841  69.260056   
2014-01-08  11.422651  151.279999  568.484192  146.351135  69.698692   

Attributes             Close                                      ...  \
Symbols           CRM      F        TSLA        GOOG         IBM  ...   
Date                                                              ...   
2014-01-02  54.860001  15.44  150.100006  554.481689  185.529999  ...   
2014-01-03  55.119999  15.51  149.559998  550.436829  186.639999  ...   
2014-01-06  54.230000  15.58  147.000000  556.573853  186.000000  ...   
2014-01-07  54.950001  15.38  149.360001  567.303589  189.710007  ...   
2014-01-08  56.939999  15.54  151.279999  568.484192  187.970001  ...   

Attributes        Open                                        Volume  \
Symbols           GOOG         IBM       AAPL        CRM           F   
Date                                                                   
2014-01-02  555.647278  187.210007  79.382858  55.200001  31528500.0   
2014-01-03  555.418152  185.830002  78.980003  54.930000  46122300.0   
2014-01-06  554.426880  187.149994  76.778572  55.200001  42657600.0   
2014-01-07  560.399475  186.389999  77.760002  54.430000  54476300.0   
2014-01-08  570.860291  189.330002  76.972855  55.189999  48448300.0   

Attributes                                                           
Symbols          TSLA       GOOG        IBM         AAPL        CRM  
Date                                                                 
2014-01-02  6188400.0  3656400.0  4546500.0   58671200.0  2730200.0  
2014-01-03  4695000.0  3345800.0  4063200.0   98116900.0  1968700.0  
2014-01-06  5361100.0  3551800.0  4067800.0  103152700.0  2532700.0  
2014-01-07  5034100.0  5124300.0  5932300.0   79302300.0  2787200.0  
2014-01-08  6163200.0  4501700.0  4603700.0   64632400.0  7036900.0  

[5 rows x 36 columns]

In [42]:
data.columns.levels

FrozenList([['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], ['F', 'TSLA', 'GOOG', 'IBM', 'AAPL', 'CRM']])

In [43]:
dfs = {
    name: data[name]
    for name in data.columns.levels[0]
}

In [45]:
dfs['Close'].head()

Symbols,F,TSLA,GOOG,IBM,AAPL,CRM
Date,,,,,,
2014-01-02,15.44,150.100006,554.481689,185.529999,79.018570,54.860001
2014-01-03,15.51,149.559998,550.436829,186.639999,77.282860,55.119999
2014-01-06,15.58,147.000000,556.573853,186.000000,77.704285,54.230000
2014-01-07,15.38,149.360001,567.303589,189.710007,77.148575,54.950001
2014-01-08,15.54,151.279999,568.484192,187.970001,77.637146,56.939999


## Writing Excel data

We can write a multi-page Excel file using an ExcelWriter:

In [51]:
!pip install xlrd openpyxl

Looking in links: /Users/rick446/src/wheelhouse
You are using pip version 19.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [52]:
with pd.ExcelWriter('./data/stocks.xlsx') as writer:
    for name, sheet in dfs.items():
        sheet.to_excel(writer, name)

## Reading Excel data

We can also read a sheet from an Excel workbook:

In [54]:
closing = pd.read_excel('./data/stocks.xlsx', 'Close', index_col='Date')
closing.head()

,F,TSLA,GOOG,IBM,AAPL,CRM
Date,,,,,,
2014-01-02,15.44,150.100006,554.481689,185.529999,79.018570,54.860001
2014-01-03,15.51,149.559998,550.436829,186.639999,77.282860,55.119999
2014-01-06,15.58,147.000000,556.573853,186.000000,77.704285,54.230000
2014-01-07,15.38,149.360001,567.303589,189.710007,77.148575,54.950001
2014-01-08,15.54,151.279999,568.484192,187.970001,77.637146,56.939999


In [55]:
closing.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1007 entries, 2014-01-02 to 2017-12-29
Data columns (total 6 columns):
F       1007 non-null float64
TSLA    1007 non-null float64
GOOG    1007 non-null float64
IBM     1007 non-null float64
AAPL    1007 non-null float64
CRM     1007 non-null float64
dtypes: float64(6)
memory usage: 55.1 KB


## Data from SQL

In [56]:
import sqlite3
con = sqlite3.connect('./data/real-estate.db')
transactions = pd.read_sql(
    'SELECT * FROM transactions', con, 
    index_col='index', 
    parse_dates=['sale_date'],
)
transactions.head()

/Users/rick446/.virtualenvs/py37/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
index,,,,,,,,,,,,
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,2008-05-21,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,2008-05-21,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,2008-05-21,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,2008-05-21,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,2008-05-21,81900,38.519470,-121.435768


We can even build a quick little bulk load function in a couple of lines of pandas:

In [57]:
stock = pd.read_csv('./data/closing-prices.csv', index_col=[0], parse_dates=True)
stock.to_sql('stock', con, if_exists='append')

In [58]:
for row in con.execute('select * from stock limit 5'):
    print(row)

('2014-01-02 00:00:00', 12.089, 150.1, None, 157.6001, 72.7741)
('2014-01-03 00:00:00', 12.1438, 149.56, None, 158.543, 71.1756)
('2014-01-06 00:00:00', 12.1986, 147.0, None, 157.9993, 71.5637)
('2014-01-07 00:00:00', 12.042, 149.36, None, 161.1508, 71.0516)
('2014-01-08 00:00:00', 12.1673, 151.28, None, 159.6728, 71.5019)


In [59]:
con.execute('select count(*) from stock').fetchall()

[(5035,)]

## Data from HTML

In [61]:
!pip install html5lib

Looking in links: /Users/rick446/src/wheelhouse
You are using pip version 19.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [62]:
tables = pd.read_html(
    'https://en.wikipedia.org/wiki/Python_(genus)',
)

In [63]:
len(tables)

7

In [64]:
tables[0].head()

,Python,Python.1
0,NaN,NaN
1,Burmese python (Python bivittatus),Burmese python (Python bivittatus)
2,Scientific classification,Scientific classification
3,Kingdom:,Animalia
4,Phylum:,Chordata


In [65]:
tables = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population',
    match='New York'
)
len(tables)

2

In [66]:
tables[0]

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York[d],New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston[3],Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W
5,6,Philadelphia[e],Pennsylvania,1584138,1526006,+3.81%,134.2 sq mi,347.6 km2,"11,683/sq mi","4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W
6,7,San Antonio,Texas,1532233,1327407,+15.43%,461.0 sq mi,"1,194.0 km2","3,238/sq mi","1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W
7,8,San Diego,California,1425976,1307402,+9.07%,325.2 sq mi,842.3 km2,"4,325/sq mi","1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°W
8,9,Dallas,Texas,1345047,1197816,+12.29%,340.9 sq mi,882.9 km2,"3,866/sq mi","1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W
9,10,San Jose,California,1030119,945942,+8.90%,177.5 sq mi,459.7 km2,"5,777/sq mi","2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°W


## Data from JSON APIs

In [67]:
!pip install requests

Looking in links: /Users/rick446/src/wheelhouse
You are using pip version 19.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [68]:
import requests

# I don't have any idea who's API key this is, but they're free, so....
APPID = '10d4440bbaa8581bb8da9bd1fbea5617'   
UNITS = 'imperial'
city = 'Atlanta'
resp = requests.get(
    f'http://api.openweathermap.org/data/2.5/forecast', 
    params={
        'q': city,
        'units': UNITS,
        'appid': APPID,
    }
)
data = resp.json()

In [69]:
data['list'][0]

{'dt': 1588377600,
 'main': {'temp': 70.43,
  'feels_like': 62.91,
  'temp_min': 69.91,
  'temp_max': 70.43,
  'pressure': 1017,
  'sea_level': 1018,
  'grnd_level': 982,
  'humidity': 31,
  'temp_kf': 0.29},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'clouds': {'all': 7},
 'wind': {'speed': 8.86, 'deg': 305},
 'sys': {'pod': 'd'},
 'dt_txt': '2020-05-02 00:00:00'}

In [71]:
# Python magic to build a list of dicts

raw_data = [
    {
        'date': row['dt_txt'], 
        **row['main'], 
        **row['weather'][0]
    } 
    for row in data['list']
]

In [72]:
raw_data[0]

{'date': '2020-05-02 00:00:00',
 'temp': 70.43,
 'feels_like': 62.91,
 'temp_min': 69.91,
 'temp_max': 70.43,
 'pressure': 1017,
 'sea_level': 1018,
 'grnd_level': 982,
 'humidity': 31,
 'temp_kf': 0.29,
 'id': 800,
 'main': 'Clear',
 'description': 'clear sky',
 'icon': '01d'}

In [73]:
weather = pd.DataFrame.from_dict(raw_data)
weather.head()

,date,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,id,main,description,icon
0,2020-05-02 00:00:00,70.43,62.91,69.91,70.43,1017,1018,982,31,0.29,800,Clear,clear sky,01d
1,2020-05-02 03:00:00,66.20,60.93,64.90,66.20,1019,1020,983,42,0.72,802,Clouds,scattered clouds,03n
2,2020-05-02 06:00:00,62.11,58.42,61.52,62.11,1021,1021,984,49,0.33,802,Clouds,scattered clouds,03n
3,2020-05-02 09:00:00,59.04,55.45,58.95,59.04,1020,1020,983,54,0.05,800,Clear,clear sky,01n
4,2020-05-02 12:00:00,59.70,56.44,59.70,59.70,1022,1022,985,50,0.00,800,Clear,clear sky,01d


In [74]:
weather['date'] = pd.to_datetime(weather['date'])
weather.set_index('date', inplace=True)
weather.head()

,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,id,main,description,icon
date,,,,,,,,,,,,,
2020-05-02 00:00:00,70.43,62.91,69.91,70.43,1017,1018,982,31,0.29,800,Clear,clear sky,01d
2020-05-02 03:00:00,66.20,60.93,64.90,66.20,1019,1020,983,42,0.72,802,Clouds,scattered clouds,03n
2020-05-02 06:00:00,62.11,58.42,61.52,62.11,1021,1021,984,49,0.33,802,Clouds,scattered clouds,03n
2020-05-02 09:00:00,59.04,55.45,58.95,59.04,1020,1020,983,54,0.05,800,Clear,clear sky,01n
2020-05-02 12:00:00,59.70,56.44,59.70,59.70,1022,1022,985,50,0.00,800,Clear,clear sky,01d


## Writing csv data

In [75]:
weather.to_csv('./data/weather.csv')

In [76]:
!head data/weather.csv

date,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,id,main,description,icon
2020-05-02 00:00:00,70.43,62.91,69.91,70.43,1017,1018,982,31,0.29,800,Clear,clear sky,01d
2020-05-02 03:00:00,66.2,60.93,64.9,66.2,1019,1020,983,42,0.72,802,Clouds,scattered clouds,03n
2020-05-02 06:00:00,62.11,58.42,61.52,62.11,1021,1021,984,49,0.33,802,Clouds,scattered clouds,03n
2020-05-02 09:00:00,59.04,55.45,58.95,59.04,1020,1020,983,54,0.05,800,Clear,clear sky,01n
2020-05-02 12:00:00,59.7,56.44,59.7,59.7,1022,1022,985,50,0.0,800,Clear,clear sky,01d
2020-05-02 15:00:00,73.11,69.21,73.11,73.11,1022,1022,986,30,0.0,800,Clear,clear sky,01d
2020-05-02 18:00:00,81.84,76.77,81.84,81.84,1021,1021,985,24,0.0,800,Clear,clear sky,01d
2020-05-02 21:00:00,82.63,76.39,82.63,82.63,1019,1019,983,27,0.0,800,Clear,clear sky,01d
2020-05-03 00:00:00,77.11,72.79,77.11,77.11,1019,1019,983,37,0.0,800,Clear,clear sky,01d


Open the [Pandas IO Lab][pandas-io]

[pandas-io]: ./pandas-io.ipynb